In [33]:
from bs4 import BeautifulSoup


In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time


In [38]:
driver = webdriver.Firefox()

In [69]:
driver.get("https://www.letemps.ch/monde?after=2024-09-25T07%3A56%3A01%2B02%3A00")

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


In [70]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [71]:
articles = soup.find_all('article', class_='sc-d12f4ae0-0 bVeLSx sc-de14d5b8-0 hUBJMX sc-3bbfd9f1-5 Kbagv')

In [72]:
# Create a list of dictionaries with titles and links
results = [
    {"title": article.find('span', class_='sc-f281f827-7 eOpKKY kilkaya-teaser-title').text, 
     "link": f"https://www.20min.ch/{article.find('a')['href']}"}
    for article in articles
]
len(results)

28